In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install gensim==3.7.3

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('queries_base_result.csv')

In [5]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['Natasha']
del df['Natasha + new rule']

In [6]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [7]:
train = df[0:1606]
test = df[1607:]

In [8]:
document_response = {}
n = 0 
for index, row in train.iterrows():
    document_response[n] = row['Номер связки\n']
    n += 1

In [9]:
df.head()

,Текст вопроса,Номер связки\n,Тематика,Без удаления NE,Deepmipt
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",уважение вероника игорь илья август год москва...,уважение август год планировать приехать гость...
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,здравствовать проинформировать пожалуйста нужн...,здравствовать проинформировать пожалуйста нужн...
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,добрый день сидельников андрей олег 30071989гр...,добрый день 1989г р посадить карантин контактн...
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",добрый день кемеровский область согласно поста...,добрый день согласно постановление необходимый...
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",здравствовать мыть город кострома ввести огран...,здравствовать мыть город ввести ограничение ра...


In [10]:
!wget http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-10-18 12:22:28--  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  10.7MB/s    in 4m 10s  

2020-10-18 12:26:40 (10.3 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [11]:
!tar --gunzip --extract --verbose --file=araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [10]:
from gensim.models import Word2Vec, KeyedVectors

In [11]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

# Классический способ

In [12]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [13]:
def word2vec_vector(text: str):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))
    # если слово есть в модели, берем его вектор
    for idx, lemma in enumerate(lemmas):
        if lemma in model:
            lemmas_vectors[idx] = model[lemma]
    
    # проверка на случай, если на вход пришел пустой массив
    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)
    
    return vec

In [14]:
def counting_top(res):
    d = {}
    # создаем словарь, где ключ - индекс документа, значение - близость документа с запросом
    index = 0
    for doc in res:
        d[index] = doc
        index += 1
    sorted_by_value = sorted(d.items(), key=lambda x: x[1], reverse=True)

    return sorted_by_value[0][0]

## Для данных без именованных сущностей

In [15]:
corpus = train['Deepmipt']
answers = test['Deepmipt']

In [16]:
corpus_vectors = np.array([normalize_vec(word2vec_vector(text)) for text in corpus])

In [17]:
answers_vectors = [normalize_vec(word2vec_vector(text)) for text in answers]

In [18]:
best_docs = []
for ans in answers_vectors:
    res = np.dot(corpus_vectors, np.array(ans))
    res = res.tolist()
    best_docs.append(counting_top(res))

In [19]:
test['deepmipt_vectors'] = [document_response[n] for n in best_docs]

## Для данных с именованными сущностями

In [20]:
corpus = train['Без удаления NE']
answers = test['Без удаления NE']

In [21]:
corpus_vectors = np.array([normalize_vec(word2vec_vector(text)) for text in corpus])

In [22]:
answers_vectors = [normalize_vec(word2vec_vector(text)) for text in answers]

In [23]:
best_docs = []
for ans in answers_vectors:
    res = np.dot(corpus_vectors, np.array(ans))
    res = res.tolist()
    best_docs.append(counting_top(res))

In [24]:
test['withNE_vectors'] = [document_response[n] for n in best_docs]

# Экспериментальный способ

In [25]:
def word2vec_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors   

In [26]:
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    return np.argmax(sims)

## Для данных без именованных сущностей

In [27]:
corpus = train['Deepmipt']
answers = test['Deepmipt']

In [28]:
corpus_matrix = np.array([word2vec_matrix(text) for text in corpus])

In [29]:
answers_matrix = [word2vec_matrix(text) for text in answers]

In [30]:
best_docs = []
for ans in answers_matrix:
    best_docs.append(search(corpus_matrix, ans))

In [31]:
test['deepmipt_matrix'] = [document_response[n] for n in best_docs]

## Для данных с именованными сущностями

In [32]:
corpus = train['Без удаления NE']
answers = test['Без удаления NE']

In [33]:
corpus_matrix = np.array([word2vec_matrix(text) for text in corpus])

In [34]:
answers_matrix = [word2vec_matrix(text) for text in answers]

In [35]:
best_docs = []
for ans in answers_matrix:
    best_docs.append(search(corpus_matrix, ans))

In [36]:
test['withNE_matrix'] = [document_response[n] for n in best_docs]

In [37]:
test.head()

,Текст вопроса,Номер связки\n,Тематика,Без удаления NE,Deepmipt,deepmipt_vectors,withNE_vectors,deepmipt_matrix,withNE_matrix
1613,Здравствуйте. меня зовут Елизавета Николаевна ...,257.0,"ОКАЗАНИЕ МЕДИЦИНСКОЙ ПОМОЩИ, ГОТОВНОСТЬ СТАЦИО...",здравствовать звать елизавета николай поздняк ...,здравствовать звать 1987г р август сообщить по...,32.0,32.0,37.0,37.0
1614,"То есть, если я пересекаю границу на машине, в...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",пересекать граница машина весь равно нужно сда...,пересекать граница машина весь равно нужно сда...,308.0,308.0,308.0,308.0
1615,"Добрый день. Подскажите, пожалуйста, нужно ли ...",308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",добрый день подсказать пожалуйста нужно турист...,добрый день подсказать пожалуйста нужно турист...,308.0,308.0,308.0,308.0
1616,Добрый день!\nМеня зовут Азиза. Я гражданка Ка...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",добрый день звать азиз гражданка казахстан про...,добрый день звать гражданка проживать супруг г...,308.0,308.0,308.0,308.0
1617,"Добрый день! Мы подрядная организация, занимае...",37.0,"ОГРАНИЧЕНИЯ, ПРОПУСКНАЯ СИСТЕМА И ПЕРЕМЕЩЕНИЕ ...",добрый день подрядный организация заниматься к...,добрый день подрядный организация заниматься к...,135.0,37.0,308.0,308.0


# Accuracy

In [38]:
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score

In [39]:
acc_withne_vectors = accuracy_score(test['Номер связки\n'], test['withNE_vectors'])
acc_deepmipt_vectors = accuracy_score(test['Номер связки\n'], test['deepmipt_vectors'])
acc_withne_matrix = accuracy_score(test['Номер связки\n'], test['withNE_matrix'])
acc_deepmipt_matrix = accuracy_score(test['Номер связки\n'], test['deepmipt_matrix'])

x = PrettyTable()
x.field_names = [' ', 'С удалением NE', 'Без удаления NE']
x.add_row(['vector', acc_deepmipt_vectors, acc_withne_vectors])
x.add_row(['matrix', acc_deepmipt_matrix, acc_withne_matrix])
print(x)

+--------+---------------------+--------------------+
|        |    С удалением NE   |  Без удаления NE   |
+--------+---------------------+--------------------+
| vector |  0.5095729013254786 | 0.5390279823269514 |
| matrix | 0.40942562592047127 | 0.4270986745213549 |
+--------+---------------------+--------------------+
